In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.imputation.mice import MICE, MICEData
from matplotlib import pyplot
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
import time
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.neural_network import MLPClassifier
import itertools
#import all the functions we wrote ourselves


In [15]:
#index_col=0 to make the companies/ the dates the indizes
sp=pd.read_csv("Data/SP500.csv",sep=",",index_col=0)
gdp = pd.read_csv("Data/GDP USA.csv", sep=",", index_col=0)
infl = pd.read_csv("Data/INFL.csv",sep=",", index_col=0)
df14=pd.read_csv("Data/2014_Financial_Data.csv",sep=",",index_col=0)
df15=pd.read_csv("Data/2015_Financial_Data.csv",sep=",",index_col=0)
df16=pd.read_csv("Data/2016_Financial_Data.csv",sep=",",index_col=0)
df17=pd.read_csv("Data/2017_Financial_Data.csv",sep=",",index_col=0)
df18=pd.read_csv("Data/2018_Financial_Data.csv",sep=",",index_col=0)

In [30]:
# create variables for gdp and inflation per year
gdp14 = gdp.iloc[1][0] * 1000000000
gdp15 = gdp.iloc[2][0] * 1000000000
gdp16 = gdp.iloc[3][0] * 1000000000
gdp17 = gdp.iloc[4][0] * 1000000000
gdp18 = gdp.iloc[5][0] * 1000000000

infl14 = infl.iloc[0][0]
infl15 = infl.iloc[1][0]
infl16 = infl.iloc[2][0]
infl17 = infl.iloc[3][0]
infl18 = infl.iloc[4][0]

In [31]:
#making one column with price var (neater if done here)
#no need for pipeline

df14.rename(columns={"2015 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df15.rename(columns={"2016 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df16.rename(columns={"2017 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df17.rename(columns={"2018 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df18.rename(columns={"2019 PRICE VAR [%]": "PRICE_VAR"},inplace=True)

In [33]:
#creating year variable

def year(data):
    all_data=[df14,df15,df16,df17,df18]
    count=0
    for data in all_data:
        data["year"]=2014+count
        count+=1
    
    #converting to cat. variable

    all_data=[df14,df15,df16,df17,df18]

    for data in all_data:
        data["year"]=data.year.astype("category")

#making a pipeline
add_year=Pipeline(steps=[("add_year",year(all_data))])

In [8]:
#creating the values with the perf. of the s&p to compare the companies to
sp14_performance=sp.loc["2014-12-31"].item()
sp15_performance=sp.loc["2015-12-31"].item()
sp16_performance=sp.loc["2016-12-30"].item()
sp17_performance=sp.loc["2017-12-29"].item()
sp18_performance=sp.loc["2018-12-31"].item()
#list with the performances
sp_performances=[sp14_performance,sp15_performance,sp16_performance,sp17_performance,sp18_performance]

In [9]:
#using encoder to make the categorical vars as that will still work when using the pipeline
from sklearn import preprocessing as pp
from sklearn.pipeline import Pipeline, FeatureUnion

all_data=[df14,df15,df16,df17,df18]

def categorize(data):

    le=pp.LabelEncoder()

    for data in all_data:
        data["Sector"]=le.fit_transform(data["Sector"])
        
#making a pipeline fot that stuff
sector_categorical=Pipeline(steps=[("sector_categorical",categorize(all_data))])

In [10]:
#combining the datasets with an outer join to make a huge df

df=pd.concat([df14,df15,df16,df17,df18])
df.shape
df

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,PRICE_VAR,Class,year
PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,1.449400e+10,...,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,3,-9.323276,0,2014
VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,1.753823e+08,...,NaN,NaN,NaN,NaN,1.6484,1.7313,3,-25.512193,0,2014
KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,2.270000e+09,...,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,3,33.118297,1,2014
RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,2.502180e+08,...,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,3,2.752291,1,2014
GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,2.707700e+09,...,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,3,12.897715,1,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YRIV,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,3.755251e+06,3.755251e+06,-3.755251e+06,1.105849e+07,-1.482451e+07,...,0.0000,-0.0508,-0.1409,-0.0152,0.0000,-0.2602,8,-90.962099,0,2018
YTEN,5.560000e+05,-0.4110,0.000000e+00,5.560000e+05,4.759000e+06,5.071000e+06,9.830000e+06,-9.274000e+06,0.000000e+00,-9.170000e+06,...,0.0000,-0.2323,-0.8602,0.0000,0.0352,-0.0993,0,-77.922077,0,2018
ZKIN,5.488438e+07,0.2210,3.659379e+07,1.829059e+07,1.652633e+06,7.020320e+06,8.672953e+06,9.617636e+06,1.239170e+06,8.416324e+06,...,0.7706,0.2489,0.4074,-0.0968,0.2415,0.8987,0,-17.834400,0,2018
ZOM,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,1.031715e+07,4.521349e+06,1.664863e+07,-1.664863e+07,0.000000e+00,-1.664769e+07,...,0.0000,0.1568,-0.2200,0.0000,2.7499,0.1457,7,-73.520000,0,2018


In [11]:

#creating train and test data in spirit of time series analysis

X_train=pd.concat([df14,df15,df16,df17])
X_test=df18

X_train.shape

(17685, 225)

In [12]:
#how many NAs are there and the percentage of missing values
df.isnull().sum().sort_values(ascending=False)/len(df)

def drop_na(data):
    #dropping the columns with more than one third of the cells missing
    df.dropna(axis="columns", thresh=(1/3)*len(df))

#pipeline
drop=Pipeline(steps=[("drop_na",drop_na(all_data))])

In [13]:
#imputer

#handling NAs with a multivariate feature imputation which essentially regresses column x_i with i=1,...N on columns X_{-i} and predicts the missing values
#takes a lot of time; does not work yet I think!

def imputer(data):
    
    imputer=IterativeImputer(max_iter=1,random_state=0)
    imputer.fit(data)
    df_filled=imputer.transform(data)

    return df_filled

imputer(df)

impute=Pipeline(steps=[("imputer",imputer(all_data))])

C:\Users\Jan Beeler\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


ValueError: setting an array element with a sequence.

In [ ]:
#outliers (saw that in the description)

# Get stocks that increased more than 200%
gain = 200
top_gainers = df[df["PRICE_VAR"] >= gain]
top_gainers = top_gainers["PRICE_VAR"].sort_values(ascending=False)
print(f"{len(top_gainers)} STOCKS with more than {gain}% gain.")

#we have 231 stocks with more than 200% gain

#we do not want these stocks as we think that they are "faked" (or typos etc.)

df = df.drop(top_gainers.index)

In [ ]:
#visualization
#outliers (saw that in the description)

#df14_outliers=df14.loc[:,["Sector", "2015 PRICE VAR [%]"]]

#sector_list=df14_outliers["Sector"].unique()

#plot the price variations per sector
#for sector in sector_list:
    
    #temp=df14_outliers[df14_outliers["Sector"] == sector]
    
    #plt.figure(figsize=(30,5))
    #plt.plot(temp["2015 PRICE VAR [%]"])
    #plt.title(sector.upper(),fontsize=20)
    #plt.show()

#price_growth=500
#gains=df14_outliers[df14_outliers["2015 PRICE VAR [%]"]>=price_growth]
#print(gains)

In [ ]:
# buy: excess return >= 2.5
# hold: 2.5 > excess return > -2.5
# sell: -2.5 >= excess return

df14.rename(columns={"2015 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df15.rename(columns={"2016 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df16.rename(columns={"2017 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df17.rename(columns={"2018 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df18.rename(columns={"2019 PRICE VAR [%]": "PRICE_VAR"},inplace=True)

all_data = [df14, df15, df16, df17, df18]

for data in all_data:
    data["buy"] = np.where((data["PRICE_VAR"].subtract(sp14_performance)) >= 2.5, [1], [0])
    data["hold"] = np.where((data["PRICE_VAR"].subtract(sp14_performance) < 2.5) &
                        (data["PRICE_VAR"].subtract(sp14_performance) > -2.5), [1], [0])
    data["sell"] = np.where((data["PRICE_VAR"].subtract(sp14_performance)) <= -2.5, [1], [0])
    

df14[["buy", "hold", "sell"]].head()

In [ ]:
#for just one column (problem with length)

all_data = [df14, df15, df16, df17, df18]
sp_performances

for data, value in zip(all_data,sp_performances):
        data["strategy"] = np.where((data["PRICE_VAR"].subtract(sp_performances)) >= 2.5, ["buy"], [""])
        data["strategy"] = np.where((data["PRICE_VAR"].subtract(sp_performances) < 2.5) &
                            (data["PRICE_VAR"].subtract(sp_performances) > -2.5), ["hold"], [""])
        data["strategy"] = np.where((data["PRICE_VAR"].subtract(sp_performances)) <= -2.5, ["sell"], [""])

In [ ]:
#evaluation function for the algos (from towards data science; might be useful later)

def evaluation(y, y_hat, title = 'Confusion Matrix'):
    cm = confusion_matrix(y, y_hat)
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    accuracy = accuracy_score(y,y_hat)
    f1 = f1_score(y,y_hat)
    print('Recall: ', recall)
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('F1: ', f1)
    sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)
    
    plt.show();

In [ ]:
#pipelines:
from sklearn.pipeline import Pipeline as pl

In [ ]:
#pipelining all of the above to prevent leakage from test data into training data which might be useful later (from tds)
def cross_validate(classifier, cv):
    pipeline = Pipeline(steps=[
        ('preprocess', preprocess),
        ('classifier', classifier)
    ])
    train_acc = []
    test_acc = []
    for train_ind, val_ind in cv.split(X_train, y_train):
        X_t, y_t = X_train.iloc[train_ind], y_train[train_ind]
        pipeline.fit(X_t, y_t)
        y_hat_t = pipeline.predict(X_t)
        train_acc.append(accuracy_score(y_t, y_hat_t))
        X_val, y_val = X_train.iloc[val_ind], y_train[val_ind]
        y_hat_val = pipeline.predict(X_val)
        test_acc.append(accuracy_score(y_val, y_hat_val))
    print(evaluation(y_val, y_hat_val))
    print('Training Accuracy: {}'.format(np.mean(train_acc)))
    print('\n')
    print('Validation Accuracy: {}'.format(np.mean(test_acc)))
    print('\n')